<a href="https://colab.research.google.com/github/Swapn2/PyTorch/blob/main/optimizing_NN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [36]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [37]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [38]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
X_train = X_train/255.0
X_test = X_test/255.0

In [45]:
y

array([2, 9, 6, ..., 8, 8, 7])

In [46]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00392157, 0.        ,
        0.        ]])

In [47]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [50]:
train_dataset = CustomDataset(X_train, y_train)

In [51]:
test_dataset = CustomDataset(X_test, y_test)

In [52]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [53]:
len(train_loader)

1500

In [54]:
# class MyNN(nn.Module):
#   def __init__(self, num_features):
#     super().__init__()
#     self.model = nn.Sequential(
#         nn.Linear(num_features, 128),
#         nn.BatchNorm1d(128),
#         nn.ReLU(),
#         nn.Dropout(p = .3),
#         nn.Linear(128,64),
#         nn.BatchNorm1d(64),
#         nn.ReLU(),
#         nn.Dropout(p =.3),
#         nn.Linear(64,10)
#     )
#   def forward(self,x):
#     return self.model(x)
class MyNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

In [55]:
learning = .01
epochs = 100

In [56]:
model = MyNN(X_train.shape[1])

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=1e-4)

In [57]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 1.4692517773707707
Epoch: 2 , Loss: 1.0206150142351786
Epoch: 3 , Loss: 0.857715164065361
Epoch: 4 , Loss: 0.7667796153227489
Epoch: 5 , Loss: 0.7110526027878126
Epoch: 6 , Loss: 0.6693947888215382
Epoch: 7 , Loss: 0.6376091431180636
Epoch: 8 , Loss: 0.617768183251222
Epoch: 9 , Loss: 0.5943423661490281
Epoch: 10 , Loss: 0.5809770137270291
Epoch: 11 , Loss: 0.5691861673295497
Epoch: 12 , Loss: 0.5578246364394823
Epoch: 13 , Loss: 0.542705706278483
Epoch: 14 , Loss: 0.530819175640742
Epoch: 15 , Loss: 0.5209216638406118
Epoch: 16 , Loss: 0.5183961814244589
Epoch: 17 , Loss: 0.5070176852941513
Epoch: 18 , Loss: 0.5039818179110686
Epoch: 19 , Loss: 0.5017035722335179
Epoch: 20 , Loss: 0.49457693720857304
Epoch: 21 , Loss: 0.4835594957371553
Epoch: 22 , Loss: 0.4805612218081951
Epoch: 23 , Loss: 0.48260102849205333
Epoch: 24 , Loss: 0.4729301788012187
Epoch: 25 , Loss: 0.4684614722530047
Epoch: 26 , Loss: 0.46789945842822395
Epoch: 27 , Loss: 0.4572704062263171
Epoch: 28 ,

In [58]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [59]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8903333333333333


In [60]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9248958333333334
